In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold, permutation_test_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [94]:
train = pd.read_csv('train.csv').fillna(99)
train.head()

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,0,3,32,3.0,99.0,323011,3854,481,1975,1,...,99.0,99,99,99,99,99,99,99,99,99
1,1,2,26,99.0,8.0,268131,2441,344,1981,1,...,99.0,99,1,99,2,99,2,99,2,99
2,2,1,16,99.0,7.0,167581,754,143,1995,1,...,1.0,99,2,99,2,99,2,99,2,99
3,3,4,44,5.0,99.0,445071,5705,604,1980,1,...,99.0,99,2,99,2,99,99,99,99,99
4,4,4,43,99.0,6.0,436161,5645,592,1958,1,...,99.0,99,1,99,1,99,1,99,1,99


In [95]:
train.shape

(18255, 1235)

In [96]:
train=train.select_dtypes(exclude=[np.object])

In [97]:
train.shape

(18255, 1139)

In [98]:
train.dtypes

train_id           int64
AA3                int64
AA4                int64
AA5              float64
AA6              float64
AA7                int64
AA14               int64
AA15               int64
DG1                int64
is_female          int64
DG3                int64
DG3A               int64
DG4                int64
DG4_OTHERS       float64
DG5_1              int64
DG5_2              int64
DG5_3              int64
DG5_4              int64
DG5_5              int64
DG5_6              int64
DG5_7              int64
DG5_8              int64
DG5_9              int64
DG5_10             int64
DG5_11             int64
DG5_96             int64
DG6                int64
DG8a               int64
DG8b               int64
DG8c               int64
                  ...   
FB27_96            int64
FB28_1           float64
FB28_2           float64
FB28_3           float64
FB28_4           float64
FB28_5           float64
FB28_6           float64
FB28_7           float64
FB28_8           float64


In [99]:
target_col = ['is_female']
inc_col = ["DG3","DG4","DG6","DG12B_1","DG12B_2","DG12C_1","DG12C_2","DG13_1","DG13_2","DG13_3","DG14","DL0","DL1",
           "DL2","DL4_5","DL4_11","DL4_15","DL4_16","DL4_17","DL4_20","DL4_21","DL4_22","DL4_23","DL5","DL7","DL15",
           "DL26_5","DL27","DL28","G2P1_5","G2P1_6","G2P1_7","G2P1_11","G2P1_12","G2P1_15","G2P1_16","MT1A","MT2",
           "MT6","MT7","MT7A","MT9","MT18_2","MT18_3","MT18_4","MT18_5","FF1","FF2","FF14_20","FF16_12","FF16_13",
           "FF16_19","FF16_20","MM13","IFI16_1","IFI16_3","IFI24","FL1","FL3","FL4","FL6_1","FL6_2","FL7_6","FL8_3",
           "FL8_6","FL8_7","FL9A","FL9B","FL9C","FL10","FL12","FL13","FL14","FL16","FL17","FL18","FB19","FB20","FB21",
           "FB24","FB25","FB27_1","FB27_2","FB27_3","FB27_4","FB27_5","FB27_6","FB27_7","FB27_8","FB27_9","FB27_96",
           "LN1A","LN1B","LN2_1","LN2_2","LN2_3","LN2_4","GN1","GN2","GN3","GN4","GN5","RI4"]
x_cols = [x for x in train.columns if x in inc_col]

In [100]:
X_train, X_test, y_train, y_test = train_test_split(train[x_cols], train['is_female'], test_size=1/7.0)

In [101]:
#standardizing the data for PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [102]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [103]:
from sklearn.decomposition import PCA

In [104]:
pca = PCA(.99)

In [105]:
pca.fit(X_train)

PCA(copy=True, n_components=0.99, whiten=False)

In [106]:
pca.n_components_

76

In [107]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [108]:
rfc = RandomForestClassifier(n_estimators = 100, class_weight = 'balanced')
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [109]:
rfc_pred = rfc.predict(X_test)

In [110]:
print confusion_matrix(y_test, rfc_pred)

[[ 858  298]
 [ 190 1262]]


In [111]:
print classification_report(y_test, rfc_pred)

             precision    recall  f1-score   support

          0       0.82      0.74      0.78      1156
          1       0.81      0.87      0.84      1452

avg / total       0.81      0.81      0.81      2608

